# Compression
Doing compression with modular addition to see how it affects interaction matrices and computation. 

## Compact experiment: bottleneck sweep for modular addition (P=64)
This notebook runs a lean experiment that trains bilinear+projection models for `(a+b) mod 64` across varying hidden/bottleneck dimensions.
It records the smallest hidden dimension that achieves perfect generalisation (100% accuracy on the full dataset) and lets you inspect interaction matrices via a slider.

In [1]:
# Compact experiment: imports and model definitions
import torch
from torch import nn
import torch.nn.functional as F
from dataclasses import dataclass
from einops import einsum
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import pickle
import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

# Bilinear layer similar to the other notebook (left/right via chunk)
class Bilinear(nn.Linear):
    def __init__(self, d_in: int, d_out: int, bias=False) -> None:
        super().__init__(d_in, 2 * d_out, bias=bias)
    def forward(self, x):
        left, right = super().forward(x).chunk(2, dim=-1)
        return left * right
    @property
    def w_l(self):
        return self.weight.chunk(2, dim=0)[0]
    @property
    def w_r(self):
        return self.weight.chunk(2, dim=0)[1]

@dataclass
class ModelConfig:
    p: int = 64
    d_hidden: int | None = None
    bias: bool = False

class Model(nn.Module):
    def __init__(self, cfg: ModelConfig):
        super().__init__()
        self.bi_linear = Bilinear(d_in=2*cfg.p, d_out=cfg.d_hidden, bias=cfg.bias)
        self.projection = nn.Linear(cfg.d_hidden, cfg.p, bias=cfg.bias)
    def forward(self, x):
        return self.projection(self.bi_linear(x))
    @property
    def w_l(self):
        return self.bi_linear.w_l
    @property
    def w_r(self):
        return self.bi_linear.w_r
    @property
    def w_p(self):
        return self.projection.weight

def init_model(p, d_hidden):
    cfg = ModelConfig(p=p, d_hidden=d_hidden, bias=False)
    return Model(cfg)

# Data generation for modular addition
def generate_modular_addition_data(P: int):
    a_vals = torch.arange(P).repeat_interleave(P)
    b_vals = torch.arange(P).repeat(P)
    a_1hot = F.one_hot(a_vals, num_classes=P).float()
    b_1hot = F.one_hot(b_vals, num_classes=P).float()
    x_1hot = torch.cat((a_1hot, b_1hot), dim=-1)
    targets = (a_vals + b_vals) % P
    return x_1hot, targets

# Create train/validation split (no separate test set)
def make_splits(x, y, train_frac=0.8, batch_size=256):
    # shuffle
    n = x.size(0)
    perm = torch.randperm(n)
    x = x[perm]
    y = y[perm]
    n_train = int(train_frac * n)
    x_train = x[:n_train]
    y_train = y[:n_train]
    x_val = x[n_train:]
    y_val = y[n_train:]
    train_ds = torch.utils.data.TensorDataset(x_train, y_train)
    val_ds = torch.utils.data.TensorDataset(x_val, y_val)
    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, (x_val, y_val)

print('Ready')

Device: cuda
Ready


Train with cross-entropy loss, having a Bilinear Layer in a Transformer architecture in mind, where the last predictive step is done via softmax. 

In [38]:
# Training utilities (lean) - now uses train and validation loaders
def train_model(model, train_loader, val_loader, epochs=500, lr=3e-3, weight_decay=1e-4):
    model = model.to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit = nn.CrossEntropyLoss()
    history = {'train_loss': [], 'val_acc': []}
    for ep in tqdm(range(epochs), desc='epochs'):
        model.train()
        losses = []
        for xb, tb in train_loader:
            xb = xb.to(device)
            tb = tb.to(device)
            logits = model(xb)
            loss = crit(logits, tb)

            opt.zero_grad()
            loss.backward()
            opt.step()
            losses.append(loss.item())
        avg_loss = sum(losses)/len(losses) if len(losses)>0 else 0.0
        history['train_loss'].append(avg_loss)

        # validation
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for xb, tb in val_loader:
                xb = xb.to(device)
                tb = tb.to(device)
                logits = model(xb)
                preds = logits.argmax(dim=-1)
                correct += (preds == tb).sum().item()
                total += tb.numel()
            val_acc = correct/total if total>0 else 0.0
        history['val_acc'].append(val_acc)
        if val_acc == 1.0:
            print(f"Grokking at epoch {ep}.")
            break
    return model, history, epochs

print('Training helper ready')

Training helper ready


In [ ]:
# Run the sweep: P=64, dims 1..64, up to 400 epochs each (stops when first perfect model found)
P = 64
x, y = generate_modular_addition_data(P)
print('Dataset size:', x.size(0))

# We'll sweep dims from no-bottleneck (d=P) down to d=1 so the slider can go P->1
dims = list(range(P, 0, -1))
max_epochs = 600
wd = 3e-4
lr = 3e-3
models_state = {}
int_mats, val_acc = {}, {}
remainders = [0, 5, 22, 42, 63]
# create splits once (same dataset across dims)
train_loader, val_loader, test_pair = make_splits(x, y, train_frac=0.75, batch_size=64)
x_test, y_test = test_pair
generalizes = True
for d in dims:
    print(f'\nTraining d_hidden={d}')
    m = init_model(P, d)
    trained, hist, used_epochs = train_model(m, train_loader, val_loader, epochs=max_epochs, 
                                             lr=lr, weight_decay=wd)
    val_acc[d] = hist['val_acc'][-1] if len(hist['val_acc'])>0 else 0.0
    if generalizes and val_acc[d] < 1.0:
        print(f"Model stops fully generalizing at bottleneck dim {d}.")
        print(f"Validation accuracy: {val_acc[d]:.3f}")
        generalizes = False
    elif not generalizes and val_acc[d] == 1.0:
        print(f"Model generalizes again at bottleneck dim {d}.")
        generalizes = True
    else:
        print(f"Validation accuracy: {val_acc[d]:.3f}")
    # save state dict on CPU to keep memory reasonable
    models_state[d] = {k: v.cpu().clone() for k, v in trained.state_dict().items()}
    # compute interaction matrices for the remainders
    mats = []
    with torch.no_grad():
        for r in remainders:
            q = einsum(trained.w_p[r].to('cpu'), trained.w_l.to('cpu'), trained.w_r.to('cpu'), 'hid, hid in1, hid in2 -> in1 in2')
            q = 0.5 * (q + q.mT)
            mats.append(q)
    int_mats[d] = torch.stack(mats, dim=0)

# Save to disk
date = datetime.datetime.now()
with open(f'sweep_results_{date.strptime("%d%m")}.pkl', 'wb') as f:
    pickle.dump({'val_accs': val_acc, 'models': models_state, 'int_mats': int_mats, 'remainders': remainders}, f)
print('Saved sweep_results.pkl')

Dataset size: 4096

Training d_hidden=64


epochs:   0%|          | 0/600 [00:00<?, ?it/s]

epochs: 100%|██████████| 600/600 [01:13<00:00,  8.15it/s]


Model stops fully generalizing at bottleneck dim 64.
Validation accuracy: 0.996

Training d_hidden=63


epochs:  43%|████▎     | 258/600 [00:32<00:42,  7.96it/s]


...

Validation accuracy: 0.419

Training d_hidden=2


epochs: 100%|██████████| 600/600 [01:11<00:00,  8.42it/s]


Validation accuracy: 0.159

Training d_hidden=1


epochs: 100%|██████████| 600/600 [01:11<00:00,  8.39it/s]


Validation accuracy: 0.026
Saved compression_sweep_results.pkl


Looking at the more compressed values (d=16), one seees that the main circuits is still visible  in the weights, but similar circuits are also visible, albeit at a smaller scale. 